<a href="https://colab.research.google.com/github/ilgiz-n/exupery/blob/main/LP_formwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решение задачи оптимальной раскладки стеновой опалубки на Python

Исходные данные и модель решения задачи написаны на основании статьи [Optimization of Vertical Formwork Layout Plans Using Mixed Integer Linear Programming](https://link.springer.com/article/10.1007/s40999-016-0090-6). 

В части 2 выполнен расчет раскладки опалубки для захваток "A" и "B" по отдельности, а в части 3 также совместный расчет для двух захваток в целях оптимизации комплектов опалубки (под повторное использование). В результате оптимизации показан экономический эффект - удешевление более, чем на 7% стоимости аренды, по сравнению со стоимостью комплекта подобранного из максимального количества опалубки (по каждому типоразмеру) из раскладок выполненных для захваток "A" и "B" по отдельности.

In [ ]:
%pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Часть 1. Быстрый старт. Расчет для отдельной стены на участке "A"

In [ ]:
from pulp import *

A = [5650, 3800, 5800, 4450]

# Создание задачи линейного программирования
prob = LpProblem("Formwork_Problem", LpMinimize)
delta = LpVariable(name="delta", lowBound=0, cat="Integer")

# Описываем переменные
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat="Integer") for i in range(1, 7)}

# Описываем целевую функцию
prob += 1550 * x[1] + 1700 * x[2] + 1775 * x[3] + 1850 * x[4] + 2250 * x[5] + 2450 * x[6]

# Описываем ограничения

prob += 300 * x[1] + 400 * x[2] + 450 * x[3] + 500 * x[4] + 750 * x[5] + 900 * x[6] == A[0] + delta
prob += delta <= 250

# Решаем задачу
results = prob.solve()

# Выводим результаты
print(f"status: {prob.status}, {LpStatus[prob.status]}")
print(f"Total cost: {prob.objective.value() * 2}")

for var in prob.variables():
    print(f"{var.name}: {var.value() * 2}")

status: 1, Optimal
Total cost: 32400.0
delta: 0.0
x1: 0.0
x2: 2.0
x3: 0.0
x4: 0.0
x5: 2.0
x6: 10.0


# Часть 2. Расчет опалубки для каждого из участков по отдельности

In [ ]:
# Участок А 
from pulp import *
import pandas as pd

# Входные параметры
# Длины стен в рабочей зоне

# Опции с угловыми элементами (алюминий и сталь, соответственно):
y1 = 300
y2 = 250

# Длина стен в зоне 'A' с плана:
walls_A = [5850, 4300, 6000, 4650]

# Рассчитываем откорректированные длины для опции y2 

d1A_y2 = walls_A[0] - y2 + 50
d2A_y2 = walls_A[1] - 2 * y2
d3A_y2 = walls_A[2] - y2 + 50
d4A_y2 = walls_A[3] - y2 + 50

# Получаем откорректированные длины
A = [d1A_y2, d2A_y2, d3A_y2, d4A_y2] 

# Данные об опалубке в формате "ширина:стоимость"
cost = {
    300: 1550, 
    400: 1700, 
    450: 1775, 
    500: 1850, 
    750: 2250, 
    900: 2450
}

# Решаем задачу линейного программирования для каждого значения A[i] из списка A
result_dict = {}
total_cost = 0
for i in range(len(A)):
    # Создаем задачу линейного программирования
    prob = LpProblem(f"Formwork_Problem{i}", LpMinimize)
    # Описываем переменные
    x = {i: LpVariable(name=f"{[*cost][i-1]}", 
                       lowBound=0, cat="Integer") for i in range(1,len(cost)+1)}
    delta = LpVariable(name="delta", lowBound=0, cat="Integer")
    # Определяем целевую функцию
    prob += (
        lpSum([*cost.values()][i-1] * x[i] for i in x),
        "Total Cost of panels for a wall",
    )
    # Добавляем ограничения
    prob += (
        lpSum([*cost][i-1] * x[i] for i in x) == A[i] + delta,
        "Total width constraint of panels for a wall [i]",
    )
    prob += delta <= 250
    # Решаем задачу
    results = prob.solve()
    # Добавляем результаты в словарь
    total_cost += prob.objective.value()
    result_dict[A[i]] = {}
    # Умножаем на 2, для подсчета с двух сторон
    result_dict[A[i]]['Cost'] = prob.objective.value() * 2
    for var in prob.variables():
        result_dict[A[i]][var.name] = var.value() * 2
        

# Выводим результаты в виде таблицы
result_table_A = pd.DataFrame(result_dict).T
result_table_A.index.name = 'Wall'
result_table_A.loc['Total']= result_table_A.sum()
print("Результаты для зоны A\n")
print(result_table_A)


Результаты для зоны A

           Cost  300  400  450  500  750   900  delta
Wall                                                 
5650    32400.0  0.0  2.0  0.0  0.0  2.0  10.0    0.0
3800    22500.0  0.0  0.0  0.0  2.0  4.0   4.0    0.0
5800    32800.0  0.0  2.0  0.0  0.0  0.0  12.0    0.0
4450    24500.0  0.0  0.0  0.0  0.0  0.0  10.0  100.0
Total  112200.0  0.0  4.0  0.0  2.0  6.0  36.0  100.0


In [ ]:
# Участок B
from pulp import *
import pandas as pd

# Входные параметры
# Длины стен в рабочей зоне

# Опции с угловыми элементами (алюминий и сталь, соответственно):
y1 = 300
y2 = 250

# Длина стен в зоне 'A' с плана:
walls_B = [5400, 4800, 4600, 6000]

# Получаем откорректированные длины для опции y2 
B = [x - y2 + 50 for x in walls_B]

# Данные об опалубке в формате "ширина:стоимость"
cost = {
    300: 1550, 
    400: 1700, 
    450: 1775, 
    500: 1850, 
    750: 2250, 
    900: 2450
}

# Решаем задачу линейного программирования для каждого значения A[i] из списка A
result_dict = {}
total_cost = 0
for i in range(len(B)):
    # Создаем задачу линейного программирования
    prob = LpProblem(f"Formwork_Problem{i}", LpMinimize)
    # Описываем переменные
    x = {i: LpVariable(name=f"{[*cost][i-1]}", 
                       lowBound=0, cat="Integer") for i in range(1,len(cost)+1)}
    delta = LpVariable(name="delta", lowBound=0, cat="Integer")
    # Определяем целевую функцию
    prob += (
        lpSum([*cost.values()][i-1] * x[i] for i in x),
        "Total Cost of panels for a wall",
    )
    # Добавляем ограничения
    prob += delta <= 250
    prob += (
        lpSum([*cost][i-1] * x[i] for i in x) == B[i] + delta,
        "Total width constraint of panels for a wall [i]",
    )
    # Решаем задачу
    results = prob.solve()
    # Добавляем результаты в словарь
    total_cost += prob.objective.value()
    result_dict[B[i]] = {}
    # Умножаем на 2, для подсчета с двух сторон
    result_dict[B[i]]['Cost'] = prob.objective.value() * 2
    for var in prob.variables():
        result_dict[B[i]][var.name] = var.value() * 2
        

# Выводим результаты в виде таблицы
result_table_B = pd.DataFrame(result_dict).T
result_table_B.index.name = 'Wall'
result_table_B.loc['Total']= result_table_B.sum()
print("Результаты для зоны B\n")
print(result_table_B)

Результаты для зоны B

           Cost  300  400  450  500  750   900  delta
Wall                                                 
5200    29000.0  0.0  0.0  0.0  0.0  2.0  10.0  100.0
4600    27000.0  0.0  0.0  0.0  4.0  0.0   8.0    0.0
4400    24500.0  0.0  0.0  0.0  0.0  0.0  10.0  200.0
5800    32800.0  0.0  2.0  0.0  0.0  0.0  12.0    0.0
Total  113300.0  0.0  2.0  0.0  4.0  2.0  40.0  300.0


### Посчитаем общую стоимость комплекта опалубки с количеством определенным из максимального количества каждого типа для каждой из захваток

In [ ]:
total_A = result_table_A.iloc[-1].drop(['Cost', 'delta'])
total_B = result_table_B.iloc[-1].drop(['Cost', 'delta'])
max_AB = pd.DataFrame([total_A, total_B]).max()
total_cost_max_AB = sum(max_AB.to_dict()[str(k)]*cost[k] for k in cost)
max_AB = pd.DataFrame({'Type':max_AB.index, 'Count':max_AB.values})
print('Комплектность опалубки из условия обеспечения потребности для \n' 
       'каждой захватки (максимальное количество каждого типоразмера \n'
        'из строки Total результатов для зоны A и B)\n')
print(max_AB.to_string(),'\n')
print(f'Общая стоимость комплекта опалубки: {total_cost_max_AB}')

Комплектность опалубки из условия обеспечения потребности для 
каждой захватки (максимальное количество каждого типоразмера 
из строки Total результатов для зоны A и B)

  Type  Count
0  300    0.0
1  400    4.0
2  450    0.0
3  500    4.0
4  750    6.0
5  900   40.0 

Общая стоимость комплекта опалубки: 125700.0


# Часть 3. Расчет для двух участков

In [ ]:
from pulp import *


workzones = ['A','B']
wall_numbers = ['1','2','3','4']

# Длина стен на участке 'A' с плана:
walls_A = [5850, 4300, 6000, 4650]

# Опции с угловыми элементами (алюминий и сталь, соответственно):
y1 = 300
y2 = 250

# Рассчитываем откорректированные длины для опции y2
y = y2 
overlap = 50
d1A_y = walls_A[0] - y + overlap
d2A_y = walls_A[1] - 2 * y
d3A_y = walls_A[2] - y + overlap
d4A_y = walls_A[3] - y + overlap

# Получаем откорректированные длины
A = [d1A_y, d2A_y, d3A_y, d4A_y]
A_enumerated = dict(zip(wall_numbers, A))

# Длина стен в зоне 'B' с плана:
walls_B = [5400, 4800, 4600, 6000]

# Получаем откорректированные длины для опции y2 
B = [x - y + overlap for x in walls_B]
B_enumerated = dict(zip(wall_numbers, B))

# Словарь со стенами на двух участках A и B 
walls_AB = {zone: {'A': A_enumerated, 'B': B_enumerated}[zone] for zone in workzones}

# Данные об опалубке в формате "ширина:стоимость"
panels = {
    300: 1550, 
    400: 1700, 
    450: 1775, 
    500: 1850, 
    750: 2250, 
    900: 2450
}

# Создаем задачу линейного программирования
prob = LpProblem(f"Formwork_two_zones", LpMinimize)

# Описываем переменные 
panels_var = [f"x{i+1}" for i in range(len(panels))]
panels_width = dict(zip(panels_var,[*panels]))
panels_cost= dict(zip(panels_var,[*panels.values()]))

x = LpVariable.dicts('panel', panels_var, lowBound=0, cat="Integer")

workzone_panels = LpVariable.dicts(
    'WZ', (workzones, x), lowBound=0, cat="Integer")

workzone_wall_panels = LpVariable.dicts(
    'WZW', (workzones, wall_numbers, x), lowBound=0, cat="Integer")

delta = LpVariable(name="delta", lowBound=0, cat="Integer")

# Определяем целевую функцию
prob += lpSum(panels_cost[i] * x[i] for i in x)

# Описываем ограничения
prob += delta <= 250

for k in workzones:
  for j in wall_numbers:
    prob += lpSum(panels_width[i] * workzone_wall_panels[k][j][i] for i in x) == (walls_AB[k][j] + delta)
    if walls_AB[k][j] ==  3800:
      prob += lpSum(panels_width[i] * workzone_wall_panels[k][j][i] for i in x)  == walls_AB[k][j]
for k in workzones:
  for j in wall_numbers:
    for i in x:
      prob += workzone_panels[k][i] <= x[i]
for k in workzones:
  for i in x:
    prob += workzone_panels[k][i] == 2 * lpSum(workzone_wall_panels[k][j][i] for j in wall_numbers)
    

# Решаем задачу
prob.solve()

# Статус решения задачи
print("Status:", LpStatus[prob.status])

# Результаты

# Каждая из переменных выводится с найденным оптимальным значением.
for v in prob.variables():
    print(v.name, "=", v.varValue)

# Оптимизированное значение целевой функции (стоимость комплекта опалубки)
Total_cost_AB_optimized = pulp.value(prob.objective)
print("Total cost = ", Total_cost_AB_optimized)

Status: Optimal
WZW_A_1_x1 = 0.0
WZW_A_1_x2 = 1.0
WZW_A_1_x3 = 0.0
WZW_A_1_x4 = 0.0
WZW_A_1_x5 = 1.0
WZW_A_1_x6 = 5.0
WZW_A_2_x1 = 0.0
WZW_A_2_x2 = 0.0
WZW_A_2_x3 = 0.0
WZW_A_2_x4 = 1.0
WZW_A_2_x5 = 2.0
WZW_A_2_x6 = 2.0
WZW_A_3_x1 = 0.0
WZW_A_3_x2 = 1.0
WZW_A_3_x3 = 0.0
WZW_A_3_x4 = 0.0
WZW_A_3_x5 = 0.0
WZW_A_3_x6 = 6.0
WZW_A_4_x1 = 0.0
WZW_A_4_x2 = 1.0
WZW_A_4_x3 = 1.0
WZW_A_4_x4 = 0.0
WZW_A_4_x5 = 0.0
WZW_A_4_x6 = 4.0
WZW_B_1_x1 = 1.0
WZW_B_1_x2 = 1.0
WZW_B_1_x3 = 0.0
WZW_B_1_x4 = 0.0
WZW_B_1_x5 = 0.0
WZW_B_1_x6 = 5.0
WZW_B_2_x1 = 0.0
WZW_B_2_x2 = 1.0
WZW_B_2_x3 = 0.0
WZW_B_2_x4 = 0.0
WZW_B_2_x5 = 2.0
WZW_B_2_x6 = 3.0
WZW_B_3_x1 = 0.0
WZW_B_3_x2 = 0.0
WZW_B_3_x3 = 1.0
WZW_B_3_x4 = 1.0
WZW_B_3_x5 = 1.0
WZW_B_3_x6 = 3.0
WZW_B_4_x1 = 0.0
WZW_B_4_x2 = 1.0
WZW_B_4_x3 = 0.0
WZW_B_4_x4 = 0.0
WZW_B_4_x5 = 0.0
WZW_B_4_x6 = 6.0
WZ_A_x1 = 0.0
WZ_A_x2 = 6.0
WZ_A_x3 = 2.0
WZ_A_x4 = 2.0
WZ_A_x5 = 6.0
WZ_A_x6 = 34.0
WZ_B_x1 = 2.0
WZ_B_x2 = 6.0
WZ_B_x3 = 2.0
WZ_B_x4 = 2.0
WZ_B_x5 = 6.0
WZ_B_x6 = 34.

### Сравним стоимость комплекта посчитанного в части 2 и 3.

In [ ]:
effect = 100*((total_cost_max_AB/Total_cost_AB_optimized)-1)
print(f'Разница в стоимости {round(effect,2)}%')

Разница в стоимости 7.12%
